In [1]:
%reset
import parcels
from parcels import ParticleFile, GeographicPolar
import numpy as np
import pandas as pd
import xarray as xr
#import matplotlib
import datetime as dati
from operator import attrgetter
from math import sqrt, pi, cos
import warnings as wr

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


## Coords are the same for all months

In [2]:
# Check that this matches correct drive
path = "E:\\OceanParcels\\"

#scen = np.array(["S0", "S1", "S2", "S3", "S4", "S5", "S6"])
#mon = np.array(["may", "jun", "jul", "aug", "sep", "oct"])

cs = xr.open_dataset(path + "data\\coordinates.nc").load()
lon_grid = cs.lon_grid.values
lat_grid = cs.lat_grid.values
dep_grid = cs.dep_grid.values
cs.close()

npart = 8 # will be squared, then doubled

# Releases 30 particles at grid of lats, lons and deps every ten hours
lon_range = np.linspace(lon_grid[4,4], lon_grid[37,38], num = npart)
lat_range = np.linspace(lat_grid[4,4], lat_grid[37,38], num = npart)

# Create a grid of particles
plons = np.repeat(lon_range, npart)
plons = np.repeat(plons, 2)
plats = np.tile(lat_range, npart)
plats = np.repeat(plats, 2)
pdeps = np.repeat(4.8, repeats = npart*npart)
pdeps = np.concatenate([pdeps, np.repeat(21.5, repeats = npart*npart)], axis = 0)

def DeleteParticle(particle, fieldset, time): 
    particle.delete() # Deletes particle if it's out of bounds

In [3]:
# Extra to account for accidental duplication
x = np.empty(shape = (3,128))
for i in range(128):
    x[:,i] = [plats[i], plons[i], pdeps[i]]
done = np.unique(x, axis = 1)
print(np.shape(done))
not_done = np.concatenate([done[2,4:36], done[2,4:36]])

plats = done[0]
plons = done[1]
pdeps = not_done

new_ps = np.empty(shape = (3,64))
new_ps[0,:] = plats
new_ps[1,:] = plons
new_ps[2,:] = pdeps

print(np.round(done[:,4:10],4))
print(np.round(new_ps[:,4:10],4))

del cs, lon_range, lat_range, new_ps, not_done, done, x

(3, 64)
[[-43.116  -43.116  -43.116  -43.116  -43.1117 -43.1117]
 [147.5548 147.5607 147.5667 147.5727 147.5309 147.5369]
 [  4.8      4.8      4.8      4.8     21.5     21.5   ]]
[[-43.116  -43.116  -43.116  -43.116  -43.1117 -43.1117]
 [147.5548 147.5607 147.5667 147.5727 147.5309 147.5369]
 [ 21.5     21.5     21.5     21.5      4.8      4.8   ]]


# All in one go!

In [4]:
scen = np.array(["S4", "S4"])
mons = np.array(["oct","sep"])

for mo in range(4):
    zarrname = "D:\\outputs2\\" + scen[mo] + "_" + mons[mo] + ".zarr"
    dfile = path + "data\\final grids\\final_" + mons[mo] + "_" + scen[mo] + "_1.nc"
    print("Starting ", zarrname)

    ds = xr.open_dataset(dfile).load()
    times_secs = ds.times_secs.values
    U_atten = ds.U_atten.values
    V_atten = ds.V_atten.values
    W = ds.W.values
    ds.close()

    # Define fieldset
    vals = {'U': U_atten, 'V': V_atten, 'W': W}

    dims = {'U': {'lon': lon_grid, 'lat': lat_grid, 'depth': dep_grid,  'time': times_secs},
            'V': {'lon': lon_grid, 'lat': lat_grid, 'depth': dep_grid,  'time': times_secs},
            'W': {'lon': lon_grid, 'lat': lat_grid, 'depth': dep_grid,  'time': times_secs}}

    fset = parcels.FieldSet.from_data(data = vals, dimensions = dims,
                      chunksize = 'auto', # allow_time_extrapolation = True,
                      gridindexingtype = 'nemo')

    del U_atten, V_atten, W, ds

    pset = parcels.ParticleSet.from_list(fieldset = fset, pclass = parcels.ScipyParticle, 
                                 lon = plons, lat = plats, depth = pdeps, 
                                 time = np.ones(npart*npart)*fset.gridset.dimrange('time')[0],
                                 repeatdt = dati.timedelta(hours = 14).total_seconds())
    pkern = pset.Kernel(parcels.AdvectionRK4_3D)
    pset.populate_indices()
    outfile = pset.ParticleFile(name = zarrname, outputdt = dati.timedelta(hours = 1))

    pset.execute(pkern, output_file = outfile, dt = dati.timedelta(minutes = 30), 
                 recovery = {parcels.ErrorCode.ErrorOutOfBounds: DeleteParticle})

    del pset, fset, pkern

Starting  D:\outputs2\S4_oct.zarr
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
INFO: Output files are stored in D:\outputs2\S4_oct.zarr.
  0%|          | 0/2677800.0 [00:00<?, ?it/s]Correct cell not found after 100000 iterations
  0%|          | 10800.0/2677800.0 [00:04<17:46, 2500.51it/s]Correct cell not found after 100000 iterations
  1%|          | 14400.0/2677800.0 [00:08<28:54, 1535.50it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
  1%|          | 14400.0/2677800.0 [

C:\ProgramData\anaconda3\envs\parcels_fin\lib\site-packages\numpy\core\numeric.py:330: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(a, fill_value, casting='unsafe')


  2%|▏         | 61200.0/2677800.0 [00:51<23:21, 1866.64it/s]Correct cell not found after 100000 iterations
  3%|▎         | 68400.0/2677800.0 [00:57<27:40, 1571.25it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
  3%|▎         | 68400.0/2677800.0 [01:10<27:40, 1571.25it/s]Correct cell not found after 100000 iterations
  3%|▎         | 72000.0/2677800.0 [01:11<1:08:47, 631.31it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
  3%|▎         | 75600.0/2677800.0 [01:18<1:16:00, 570.63it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
  3%|▎         | 75600.0/2677800.0 [01:30<1:16:00, 570.63it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
  3%|▎         | 79200.0/2677800.0 [01:35<1:51:08, 389.66it/s]Corre

 11%|█         | 291600.0/2677800.0 [07:40<43:05, 922.80it/s]Correct cell not found after 100000 iterations
 11%|█         | 295200.0/2677800.0 [07:43<1:20:46, 491.63it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 12%|█▏        | 313200.0/2677800.0 [07:57<32:50, 1200.28it/s] Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 12%|█▏        | 316800.0/2677800.0 [08:04<46:27, 846.98it/s] Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 12%|█▏        | 320400.0/2677800.0 [08:11<57:43, 680.74it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 14%|█▍        | 374400.0/2677800.0 [08:36<13:17, 2887.42it/s] Correct cell not found after 100000 iterations
 14%|█▍        | 378000.0/2677800.0 [08:41<24:04, 1591.98it/s]Correct cell not found after 100000 iterations
 14%|█▍        | 381600.0/2677800.0 [08:46<32:05

Correct cell not found after 100000 iterations
 20%|█▉        | 532800.0/2677800.0 [14:59<4:03:12, 146.99it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 20%|██        | 536400.0/2677800.0 [15:06<3:11:35, 186.28it/s]Correct cell not found after 100000 iterations
 20%|██        | 543600.0/2677800.0 [15:11<1:44:24, 340.68it/s]Correct cell not found after 100000 iterations
 20%|██        | 547200.0/2677800.0 [15:15<1:25:11, 416.80it/s]Correct cell not found after 100000 iterations
 21%|██        | 558000.0/2677800.0 [15:21<40:45, 866.89it/s]  Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 21%|██        | 561600.0/2677800.0 [15:29<49:41, 709.85it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 21%|██        | 561600.0/2677800.0 [15:40<49:41, 709.85it/s

...

 70%|███████   | 1825200.0/2591400.0 [53:22<55:00, 232.18it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 70%|███████   | 1825200.0/2591400.0 [53:34<55:00, 232.18it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 71%|███████   | 1828800.0/2591400.0 [53:38<55:57, 227.13it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 71%|███████   | 1832400.0/2591400.0 [53:53<54:07, 233.71it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 71%|███████   | 1832400.0/2591400.0 [54:04<54:07, 233.71it/s]Correct cell not found after 100000 iterations
 71%|███████   | 1836000.0/2591400.0 [54:07<52:04, 241.80it/s]Correct cell not 

 79%|███████▉  | 2044800.0/2591400.0 [1:00:46<13:49, 659.08it/s]Correct cell not found after 100000 iterations
 79%|███████▉  | 2048400.0/2591400.0 [1:00:50<12:40, 713.81it/s]Correct cell not found after 100000 iterations
 79%|███████▉  | 2052000.0/2591400.0 [1:00:55<12:08, 739.93it/s]Correct cell not found after 100000 iterations
 80%|███████▉  | 2062800.0/2591400.0 [1:01:01<06:47, 1298.36it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 80%|███████▉  | 2066400.0/2591400.0 [1:01:12<12:28, 701.44it/s] Correct cell not found after 100000 iterations
 80%|████████  | 2073600.0/2591400.0 [1:01:18<09:30, 907.69it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 80%|████████  | 2073600.0/2591400.0 [1:01:34<09:3

 90%|████████▉ | 2329200.0/2591400.0 [1:07:55<09:29, 460.68it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 90%|█████████ | 2332800.0/2591400.0 [1:08:15<20:09, 213.80it/s]Correct cell not found after 100000 iterations
 90%|█████████ | 2340000.0/2591400.0 [1:08:22<11:34, 362.14it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 90%|█████████ | 2343600.0/2591400.0 [1:08:29<10:38, 387.98it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 91%|█████████ | 2350800.0/2591400.0 [1:08:40<07:37, 525.49it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 92%|█████████▏| 23724

Correct cell not found after 100000 iterations
 99%|█████████▉| 2570400.0/2591400.0 [1:15:15<00:32, 652.26it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 99%|█████████▉| 2574000.0/2591400.0 [1:15:20<00:45, 385.97it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 99%|█████████▉| 2577600.0/2591400.0 [1:15:34<00:40, 341.97it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
100%|█████████▉| 2581200.0/2591400.0 [1:15:41<00:27, 376.07it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
100%|█████████▉| 2584800.0/2591400.0 [1:15:51<00:17, 370.54it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
100%|█████████▉| 25884

type: index 2 is out of bounds for axis 0 with size 2

In [ ]:
%%script echo Skipped!

import sys, glob

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)